In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [128]:
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
train_dataset = train_dataset.drop(['Cabin','Name','Ticket','PassengerId'],axis = 1)
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [129]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(train_dataset.iloc[:,3:4])
train_dataset.iloc[:,3:4] = imputer.transform(train_dataset.iloc[:,3:4])
train_dataset = train_dataset.loc[train_dataset['Embarked'].notna()]
X_train = train_dataset.iloc[:,1:]
Y_train = train_dataset.iloc[:,0]

In [ ]:
X_train['Age'] = X_train['Age'].fillna(0)
X_train['Fare'] = X_train['Fare'].fillna(0)

In [130]:
X_train.corrwith(Y_train)

Pclass   -0.335549
Age      -0.069822
SibSp    -0.034040
Parch     0.083151
Fare      0.255290
dtype: float64

In [131]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    889 non-null    int64  
 1   Sex       889 non-null    object 
 2   Age       889 non-null    float64
 3   SibSp     889 non-null    int64  
 4   Parch     889 non-null    int64  
 5   Fare      889 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.6+ KB


In [132]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
X_train = X_train.values
Y_train = Y_train.values
ct_embarked = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[6,1])],remainder='passthrough')
X_train = np.array(ct_embarked.fit_transform(X_train))
X_train

array([[0.0, 0.0, 1.0, ..., 1, 0, 7.25],
       [1.0, 0.0, 0.0, ..., 1, 0, 71.2833],
       [0.0, 0.0, 1.0, ..., 0, 0, 7.925],
       ...,
       [0.0, 0.0, 1.0, ..., 1, 2, 23.45],
       [1.0, 0.0, 0.0, ..., 0, 0, 30.0],
       [0.0, 1.0, 0.0, ..., 0, 0, 7.75]], dtype=object)

In [46]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,[2,6]] = sc.fit_transform(X_train[:,[2,6]])

In [133]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size = 0.2, random_state = 1)

In [170]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 1000,max_depth=5,random_state = 1)
rf_classifier.fit(x_train,y_train)
y_pred = rf_classifier.predict(x_test)
accuracy_score(y_test,y_pred)

0.8370786516853933

In [161]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
estimators = [('knn',KNeighborsClassifier(n_neighbors = 5)),
              ('svc',SVC(kernel='rbf')),
              ('lr',LogisticRegression())]
st_classifier = StackingClassifier(estimators = estimators, final_estimator = rf_classifier,n_jobs = -1)
st_classifier.fit(x_train,y_train)
y_pred = st_classifier.predict(x_test)
accuracy_score(y_test,y_pred)

0.8426966292134831

In [150]:
X_test = pd.read_csv('test.csv')
X_test = X_test.drop(['PassengerId','Name','Cabin','Ticket'],axis =1)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [153]:
X_test = pd.read_csv('test.csv')
X_test = X_test.drop(['PassengerId','Name','Cabin','Ticket'],axis =1)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(X_test.iloc[:,2:3])
X_test.iloc[:,2:3] = imputer.transform(X_test.iloc[:,2:3])
imputer_fare = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer_fare.fit(X_test.iloc[:,5:6])
X_test.iloc[:,5:6] = imputer_fare.transform(X_test.iloc[:,5:6])
#X_test.iloc[:,[6,2]] = sc.transform(X_test.iloc[:,[6,2]])
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [154]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [156]:
X_test = np.array(ct_embarked.transform(X_test))

In [162]:
rf_classifier.fit(X_train,Y_train)
y_rf_pred_entropy = rf_classifier.predict(X_test)

In [166]:
st_classifier.fit(X_train,Y_train)
y_pred_st = st_classifier.predict(X_test)

In [167]:
test_csv = pd.read_csv('test.csv')
result_df = pd.DataFrame(zip(test_csv['PassengerId'].values,y_rf_pred_entropy),columns=['PassengerId','Survived'])
result_df.to_csv('prediction_by_rf.csv',index=False)

In [165]:
test_csv = pd.read_csv('test.csv')
result_df = pd.DataFrame(zip(test_csv['PassengerId'].values,y_pred_st),columns=['PassengerId','Survived'])
result_df.to_csv('prediction_by_Stacking.csv',index=False)